# `PyLammpsMPI`

## Running on queueing systems

`pylammpsmpi` integrates with [dask distributed](https://distributed.dask.org/en/latest/) and [dask jobqueue](https://jobqueue.dask.org/en/latest/) to enable running lammps interactively on queueing systems. 

Dask distributed provides a local single machine cluster on which pylammpsmpi can be run. More documentation about dask single cluster is available [here](https://docs.dask.org/en/latest/setup/single-distributed.html).

In [1]:
from dask.distributed import Client, LocalCluster

Create a cluster locally with one worker and two threads per worker.

In [2]:
cluster = LocalCluster(n_workers=1, threads_per_worker=2)

/home/sarath/miniconda3/envs/lammps/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Create a client and connect to the cluster

In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39107 Dashboard: http://127.0.0.1:36385/status,Cluster Workers: 1 Cores: 2 Memory: 8.08 GB


A visual dashboard of jobs running on the cluster is also available. Once a cluster is set up, it can be provided to pylammpsmpi

In [5]:
from pylammpsmpi import LammpsLibrary

Specify two cores for the job(since we created a cluster with 2), choose the `mode` as `dask`, and pass the `client` to pylammpsmpi object

In [6]:
lmp = LammpsLibrary(cores=2, mode='dask', client=client)

The rest is similar as you would run on a local machine - except the calculations are run on the cluster.

Read an input file 

In [3]:
lmp.file("tests/in.simple")

Check version of lammps

In [4]:
lmp.version

20190807

Check number of atoms

In [5]:
lmp.natoms

256

### Run commands

In [6]:
lmp.command("run 1")

In [7]:
lmp.command(["run 1", "run 1"])

Commands can also be direct

In [8]:
lmp.run(10)

In [4]:
lmp.mass(1, 20)

### Extract a global property

In [8]:
lmp.extract_global("boxxhi", 1)

6.718384765530029

### Access thermo quantities

In [7]:
lmp.get_thermo("temp")

array(1.12985322)

Thermo quantities can also be accessed directly,

In [8]:
lmp.temp

array(1.12985322)

In [9]:
lmp.press

array(-2.60581752)

### Accessing simulation box

In [9]:
lmp.extract_box()

([0.0, 0.0, 0.0],
 [6.718384765530029, 6.718384765530029, 6.718384765530029],
 0.0,
 0.0,
 0.0,
 [1, 1, 1],
 0)

### Accessing and changing atom properties

Get individual atom properties, for example force on each atoms

In [10]:
ff = lmp.gather_atoms("f")
print(type(ff))
print(len(ff))

<class 'numpy.ndarray'>
256


Get atom properties by their ids

In [11]:
ids = lmp.gather_atoms("id")

In [12]:
ff = lmp.gather_atoms("f", ids=ids[:10])
len(ff)

10

Change atom properties

In [13]:
ff = ff*0.5
lmp.scatter_atoms("f", ff, ids=ids[:10])

### Access value of variables

In [6]:
temp = lmp.extract_variable("tt", "all", 0)
temp

0.8846341461467611

### Access value of computes

In [7]:
ke = lmp.extract_compute("ke", 1, 1)
len(ke)

256

In [8]:
v = lmp.extract_compute("v", 1, 2, width=3)
v.shape

(256, 3)

In [9]:
lmp.extract_compute("1", 0, 0)

0.8846341461467611

In [10]:
msd = lmp.extract_compute("msd", 0, 1, length=4)
msd[0]

0.005507481618069701

### Access values from fix

In [11]:
x = lmp.extract_fix("2", 0, 1, 1)
x

-2.605817524153117

### Change the simulation box

In [12]:
lmp.reset_box([0.0,0.0,0.0], [8.0,8.0,8.0], 0.0,0.0,0.0)